In [ ]:
#!/usr/bin/env python

import urllib
import json
import os

from flask import Flask
from flask import request
from flask import make_response

import pandas as pd
import json
df = pd.read_csv('./companies.csv')

# Flask app should start in global layout
app = Flask(__name__)


@app.route('/webhook', methods=['POST'])
def webhook():
    req = request.get_json(silent=True, force=True)

    print("Request:")
    print(json.dumps(req, indent=4))

    res = makeWebhookResult(req)

    res = json.dumps(res, indent=4)
    print(res)
    r = make_response(res)
    r.headers['Content-Type'] = 'application/json'
    return r

def findByName(name):
    idx = [x.find(name)>-1 for x in df['name'].str.lower()]
    return df[idx]

def searchNames(name):
    names_df = findByName(name)
    if names_df.empty: 
        print("No match found. Try again!")
    #for name in names_df['name']:
    #    print(name)
    s = makeQuickReply('Looking for:', names_df['name'])
    print(s)
    return s

def replace_right(source, target, replacement, replacements=None):
    return replacement.join(source.rsplit(target, replacements))

def makeQuickReply(text, arr):
    if len(arr) >5: 
        text = text + " (Showing only first 5 results)"
        arr = arr[:5]
    s = ''
    prefix = '''
            {
            "facebook": {
                "text": "%s",
                "quick_replies": [
                '''%text
    postfix = '''
                    ]
            }
        }
    '''
    s = s + prefix
    for name in arr: 
        s = s +'''
                        {
                           "content_type": "text",
                           "title": "''' + name + '''", 
                           "payload":"''' + name + '''"
                        },
                        '''
    s = replace_right(s, ",", "", 1)
    s = s + postfix
    #s = "".join(s.split())
    
    return s


def makeWebhookResult(req):
    result = req.get("result")
    
    if result is None: 
        print('result not found. ')
        return {}
    
    intent = result.get("metadata").get("intentName")
    parameters = result.get("parameters")
    
    if intent == 'find_company':
        name = parameters.get("search_company")
        s = "Search " + name + ":"
        return {
            "speech": s,
            "displayText": s,
            "data":json.loads(searchNames(name))
        }
    

    parameters = result.get("parameters")
    zone = parameters.get("shipping-zone")

    cost = {'Europe':100, 'North America':200, 'South America':300, 'Asia':400, 'Africa':500}

    speech = "The cost of shipping to " + zone + " is " + str(cost[zone]) + " euros."

    print("Response:")
    print(speech)

    return {
        "speech": speech,
        "displayText": speech,
        "data": {
            "facebook": {
                "text": "Pick a color:",
                "quick_replies": [
                    {
                       "content_type": "text",
                       "title": "Red",
                       "payload": "red apple"
                    },
                    {
                       "content_type": "text",
                       "title": "Green",
                       "payload": "green candy"
                    }
                ]
            }
        },
        # "contextOut": [],
        "source": "apiai-onlinestore-shipping"
    }


if __name__ == '__main__':
    port = int(os.getenv('PORT', 5000))

    print("Starting app on port %d"%port)

    app.run(debug=True, port=port, host='0.0.0.0')
